In [ ]:
import mne  # For EEG/MEG data processing
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from mne.io import read_raw_edf, read_raw_fif, read_raw_bdf
from mne.preprocessing import ICA
from tqdm import tqdm
import pickle
import mat73
from scipy.io import loadmat

# Utils

In [ ]:
def match_with_tolerance(list1, list2, tol):
    """
    For each element in list1, find a unique element in list2 such that
    abs(list1[i] - list2[j]) <= tol. If multiple candidates exist,
    pick the one with minimal |difference|. Each element in list2 can
    be matched at most once.

    Returns a list `matches` of length len(list1), where matches[i] is
    the index j in list2 matched to list1[i], or None if no match.
    """
    matches = [None] * len(list1)
    used    = set()  # keep track of already-matched indices in list2

    for i, x in enumerate(list1):
        best_j    = None
        best_diff = tol + 1e-12

        # scan through list2 to find the closest unused candidate
        for j, y in enumerate(list2):
            if j in used:
                continue
            diff = abs(x - y)
            if diff <= tol and diff < best_diff:
                best_diff = diff
                best_j    = j

        if best_j is not None:
            used.add(best_j)
        matches[i] = best_j

    return matches

# Load data and basic informations

In [ ]:
subj = '014'
#breaking = True
#reverse = not breaking

In [ ]:
# Path to the .vhdr file (make sure all three files are in the same directory)
'''
### Breaking ###
if breaking:
    file_path = 'D:/PhD/CFS_eeg/data/new_exp/task/breaking/subj_'+subj+'.vhdr' # The .vhdr file points to the others
### Reverse ###
elif reverse:
    file_path = 'D:/PhD/CFS_eeg/data/new_exp/task/reverse/revsubj_'+subj+'.vhdr' # The .vhdr file points to the others
'''

file_path = '/home/lunis/Documents/nlin-EEG/data/BMNP/subj' + subj + '.vhdr'

# Load the EEG data from the .vhdr file
raw = mne.io.read_raw_brainvision(file_path, preload=True)

# Print general information about the loaded data
print(raw.info)

# # Plot the raw EEG data
# raw.plot()

In [ ]:
# Drop Iz
raw.drop_channels(['Iz'])

In [ ]:
# for ch_name, ch_type in zip(raw.info['ch_names'], raw.get_channel_types()):
#     print(f"Channel: {ch_name}, Type: {ch_type}")

In [ ]:
raw.plot(n_channels=62);

In [ ]:
# Sampling rate (Hz)
print(f'Sampling rate: {raw.info["sfreq"]} Hz')

# List of channels (e.g., Fp1, Fp2, Cz, etc.)
print(f'Channels: {raw.ch_names}')

# Duration of the recording (in seconds)
duration = raw.n_times / raw.info['sfreq']
print(f'Recording duration: {duration} seconds')

# Number of EEG channels
n_channels = len(raw.ch_names)
print(f'Number of channels: {n_channels}')

In [ ]:
# # Iz position
# electrode_index = raw.info['ch_names'].index('Iz')

# Preprocessing

## Basic filtering and rereferencing

In [ ]:
# Filter the EEG data (remove frequencies below 1 Hz and above 40 Hz)
raw_filtered = raw.copy().filter(l_freq=0.5, h_freq=40.0)

In [ ]:
# Re-referencing

# # # Re-reference the EEG to the average of all channels
# raw_filtered.set_eeg_reference('average', projection=False)

# # Re-reference the EEG according to a reference electrode
# raw_filtered.set_eeg_reference(['Cz'])

# # Visualize the filtered data
# # raw_filtered.plot()

In [ ]:
raw_filtered.plot(n_channels=62);

## ICA preprocessing

In [ ]:
# raw_filtered

In [ ]:
ica = ICA(n_components=30, random_state=42, max_iter='auto')
ica.fit(raw_filtered)

In [ ]:
ica.plot_components(show_names=True);

In [ ]:
ica.plot_sources(raw_filtered, picks=[i for i in range(15)]);
ica.plot_sources(raw_filtered, picks=[i for i in range(15,30)]);

In [ ]:
### Print Power spectrum of ICA decomposition ###
sources = ica.get_sources(raw_filtered)

# 5. Plot the power spectrum for each ICA component
n_components = ica.n_components_

# Loop through each ICA component
for i in range(n_components):
    # Extract the signal of component i
    component_data = sources.get_data(picks=[i])  # Get data for the i-th ICA component

    # Compute the power spectral density (PSD) of the component
    psd, freqs = mne.time_frequency.psd_array_welch(
        component_data[0],  # Extract the first row (since it's a single component)
        sfreq=raw.info['sfreq'],  # Sampling frequency from the raw data
        fmin=1, fmax=40,  # Focus on the 1-50 Hz range
        n_fft=2048  # Length of FFT (controls frequency resolution)
    )

    # Plot the power spectrum of the component
    plt.figure(figsize=(5, 3))
    plt.plot(freqs, 10 * np.log10(psd), label=f'Component {i}')
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Power (dB)')
    plt.title(f'Power Spectrum of ICA Component {i}')
    plt.grid(True, linestyle='--', alpha=0.6)
    plt.show()

In [ ]:
# muscle_idx_auto, scores = ica.find_bads_muscle(raw)
# ica.plot_properties(raw, picks=muscle_idx_auto, log_scale=True)
# ica.plot_scores(scores, exclude=muscle_idx_auto)

# print(
#     f"Manually found muscle artifact ICA components:      {muscle_idx}\n"
#     "Automatically found muscle artifact ICA components: "
#     f"{muscle_idx_auto}"
# )

### Remove bad components

In [ ]:
# Manual detection of bad ICA components
ica.exclude = [0,1,2]

In [ ]:
# # Automatic detection of bad ICA components
# raw_filtered.set_channel_types({'Fp1': 'eog', 'Fp2': 'eog'})
# eog_indices, eog_scores = ica.find_bads_eog(raw_filtered)
# eog_indices

In [ ]:
cleaned_raw = raw_filtered.copy()
ica.apply(cleaned_raw);

In [ ]:
print('PRE bad IC removal')
raw_filtered.plot(n_channels=61);

In [ ]:
print('POST bad IC removal')
cleaned_raw.plot(n_channels=61);

In [ ]:
# Remove and interpole bad electrode
cleaned_raw.info['bads'] = ['TP9','TP10','T7', 'T8', 'P7', 'P8']
cleaned_raw.interpolate_bads()
cleaned_raw.plot(n_channels=61);

# Load the matlab output
### Extract informations about the matched face-noface trials

In [ ]:
### Breaking ###
if breaking:
    file_path = 'D:/PhD/CFS_eeg/data/new_exp/task/mat_breaking/bCFS_EEGfacenoface_'+subj+'.mat' # The .vhdr file points to the others
### Reverse ###
elif reverse:
    file_path = 'D:/PhD/CFS_eeg/data/new_exp/task/mat_reverse/revCFS_EEGfacenoface_'+subj+'.mat' # The .vhdr file points to the others

In [ ]:
mat_out = loadmat(file_path)
time_adjuster = mat_out['p'][0][0][5][0]*1000
# matched_trials = mat_out['p'][0][0][-1][0]
time_adjuster

In [ ]:
subj

# Extract the events

In [ ]:
# Extract event markers
events, event_dict = mne.events_from_annotations(raw) # events : n_events x ? x event code

# Print all the event names and codes
print(f'Event dictionary: {event_dict}')

# Extract epochs around a specific event (e.g., "Stimulus/S1")
# event_id = {'Stimulus/S21': 21}  # The event of interest (this may vary depending on your file)
# epochs = mne.Epochs(raw, events, event_id=event_id, tmin=-0.2, tmax=0.5, baseline=(None, 0), preload=True)

# # Plot the epochs
# epochs.plot()

In [ ]:
# 1 e 2 è inizio trial con trial vero e controllo 
# 21-22 sono accesso/scomparsa di trial vero e controllo 
# 31 32 fine trial di trial vero e controllo 
# 41-44 e 51-54 sono le confidenze, di trial vero e controllo 

In [ ]:
events[:40,:]

In [ ]:
event_dict

In [ ]:
# start_e = 1001 # code for the start of a netural trial
# start_n = 1002 # code for the start of an emotion trial
# resp_e = 1005
# resp_n = 1005
# end_e = 1007
# end_n = 1006
# confs = [41,42,43,44,51,52,53,54]
# cross_switch = 5

In [ ]:
start_e = 1 # code for the start of a netural trial
start_n = 2 # code for the start of an emotion trial
resp_e = 21
resp_n = 22
end_e = 31
end_n = 32
confs = [41,42,43,44,51,52,53,54]
cross_switch = 5

In [ ]:
# # neutral trial
# # The control time (the time of the switch of the cross) as to be matched with the response time of the face trials
# data = cleaned_raw
# # data = raw_filtered


# cond_codes = [(start_e, resp_e, end_e),(start_n, resp_n, end_n)]

# trials_dict = {'trial': [], 'trial_norm': [], 'baseline':[],'baseline_norm':[], 'resp_point': [], 'label': [], 'confidence':[], 'crosstime':[],
#                'matching':[]}
# for c, cond in enumerate(['real','contr']):# Loop over the two classes
#     print(cond)
#     start = cond_codes[c][0]
#     resp = cond_codes[c][1]
#     end = cond_codes[c][2]
#     where_start = np.where(events[:,2]==start)
#     for w in tqdm(where_start[0]): # Loop over the start of each trial
#         start_point = events[w][0]
#         i=1
#         itsended = False # the current trial has an end
#         control=False
#         resp_point=0
#         conf_val = None
#         control_time = None
#         while(w+i!=len(events)): # Loop over the successive sample after the start, looking for the end
#             curr_samp = events[w+i]
#             ##### DECOMMENTARE CON I TRIGGER NORMALI ####
#             # if curr_samp[2]==cross_switch and c==1:
#             #     # control = True
#             #     control_time = curr_samp[0]
#             if curr_samp[2]==start_n or curr_samp[2]==start_e: # The end of the trial is missing, discard the trial
#                 break
#             if curr_samp[2]==resp:
#                 if resp_point==0:
#                     itsended=True
#                     resp_point=curr_samp[0]
#                     end_point=curr_samp[0]
#             if curr_samp[2]==end:
#                 end_point=curr_samp[0]
#                 itsended=True
#             if curr_samp[2] in confs:
#                 conf_val = curr_samp[-1]
#                 break
#             i+=1

#         if itsended:
#             resp_point = resp_point-start_point
#             if c==0: # Face
#                 if resp_point>0:
#                     # I take the activity from -150 to -50 as a baseline (no stimulation)
#                     extracted_data = data.get_data(start=start_point-100, stop=end_point)
                    
#                     base = extracted_data[:, :100]
#                     # print(base.shape)
#                     mean_base = base.mean(axis=1)
#                     extracted_data_scale = extracted_data.T-mean_base
#                     # print(extracted_data_scale.shape)
#                     extracted_data_norm = extracted_data_scale/extracted_data_scale.std() # z-score the trial
#                     extracted_data_scale = extracted_data_scale.T
#                     extracted_data_norm = extracted_data_norm.T
                    
#                     trials_dict['trial'].append(extracted_data_scale[:, 100:])
#                     trials_dict['trial_norm'].append(extracted_data_norm[:, 100:])
#                     trials_dict['baseline'].append(extracted_data_scale[:, :100])
#                     trials_dict['baseline_norm'].append(extracted_data_norm[:, :100])
#                     trials_dict['resp_point'].append(resp_point)
#                     trials_dict['label'].append(cond)
#                     trials_dict['confidence'].append(conf_val)
#                     trials_dict['crosstime'].append(None)
#             elif c==1: # Control
#                 if resp_point>0:
#                     # I take the activity from -150 to -50 as a baseline (no stimulation)
#                     extracted_data = data.get_data(start=start_point-100, stop=end_point)
                    
#                     base = extracted_data[:, :100]
#                     # print(base.shape)
#                     mean_base = base.mean(axis=1)
#                     extracted_data_scale = extracted_data.T-mean_base
#                     # print(extracted_data_scale.shape)
#                     extracted_data_norm = extracted_data_scale/extracted_data_scale.std() # z-score the trial
#                     extracted_data_scale = extracted_data_scale.T
#                     extracted_data_norm = extracted_data_norm.T
#                     ##### DECOMMENTARE CON I TRIGGER NORMALI ####
#                     # control_time = control_time-start_point +time_adjuster
#                     # if control_time[0]>0:
#                     ##### DEINDENTARE CON I TRIGGER NORMALI #####
#                     trials_dict['trial'].append(extracted_data_scale[:, 100:])
#                     trials_dict['trial_norm'].append(extracted_data_norm[:, 100:])
#                     trials_dict['baseline'].append(extracted_data_scale[:, :100])
#                     trials_dict['baseline_norm'].append(extracted_data_norm[:, :100])
#                     trials_dict['resp_point'].append(resp_point)
#                     trials_dict['label'].append(cond)
#                     trials_dict['confidence'].append(conf_val)
#                     # trials_dict['crosstime'].append(control_time[0])
#                     trials_dict['crosstime'].append('cane')

                        
                
    
    
# print(len(trials_dict['trial']),len(trials_dict['trial_norm']),len(trials_dict['baseline']),len(trials_dict['baseline_norm']),
#       len(trials_dict['resp_point']), len(trials_dict['label']),len(trials_dict['confidence']),)
# print(trials_dict['trial'][0].shape,trials_dict['trial_norm'][0].shape,trials_dict['baseline'][0].shape,trials_dict['baseline_norm'][0].shape)
            


#------------- QUELLO VERO DA DECOMMENTARE CON I TRIGGER GIUSTI -------------------------------------#

# neutral trial
# The control time (the time of the switch of the cross) as to be matched with the response time of the face trials
data = cleaned_raw
# data = raw_filtered


cond_codes = [(start_e, resp_e, end_e),(start_n, resp_n, end_n)]

trials_dict = {'trial': [], 'trial_norm': [], 'baseline':[],'baseline_norm':[], 'resp_point': [], 'label': [], 'confidence':[], 'crosstime':[],
               'matching':[]}
for c, cond in enumerate(['real','contr']):# Loop over the two classes
    print(cond)
    start = cond_codes[c][0]
    resp = cond_codes[c][1]
    end = cond_codes[c][2]
    where_start = np.where(events[:,2]==start)
    for w in tqdm(where_start[0]): # Loop over the start of each trial
        start_point = events[w][0]
        i=1
        itsended = False # the current trial has an end
        control=False
        resp_point=0
        while(w+i!=len(events)): # Loop over the successive sample after the start, looking for the end
            curr_samp = events[w+i]
            if curr_samp[2]==cross_switch and c==1:
                # control = True
                control_time = curr_samp[0]
            if curr_samp[2]==start_n or curr_samp[2]==start_e: # The end of the trial is missing, discard the trial
                break
            if curr_samp[2]==resp:
                if resp_point==0:
                    itsended=True
                    resp_point=curr_samp[0]
                    end_point=curr_samp[0]
            if curr_samp[2]==end:
                end_point=curr_samp[0]
                itsended=True
            if curr_samp[2] in confs:
                conf_val = curr_samp[-1]
                break
            i+=1

        if itsended:
            resp_point = resp_point-start_point
            if c==0: # Face
                if resp_point>0:
                    # I take the activity from -150 to -50 as a baseline (no stimulation)
                    extracted_data = data.get_data(start=start_point-100, stop=end_point)
                    
                    base = extracted_data[:, :100]
                    # print(base.shape)
                    mean_base = base.mean(axis=1)
                    extracted_data_scale = extracted_data.T-mean_base
                    # print(extracted_data_scale.shape)
                    extracted_data_norm = extracted_data_scale/extracted_data_scale.std() # z-score the trial
                    extracted_data_scale = extracted_data_scale.T
                    extracted_data_norm = extracted_data_norm.T
                    
                    trials_dict['trial'].append(extracted_data_scale[:, 100:])
                    trials_dict['trial_norm'].append(extracted_data_norm[:, 100:])
                    trials_dict['baseline'].append(extracted_data_scale[:, :100])
                    trials_dict['baseline_norm'].append(extracted_data_norm[:, :100])
                    trials_dict['resp_point'].append(resp_point)
                    trials_dict['label'].append(cond)
                    trials_dict['confidence'].append(conf_val)
                    trials_dict['crosstime'].append(None)
            elif c==1: # Control
                if resp_point>0:
                    # I take the activity from -150 to -50 as a baseline (no stimulation)
                    extracted_data = data.get_data(start=start_point-100, stop=end_point)
                    
                    base = extracted_data[:, :100]
                    # print(base.shape)
                    mean_base = base.mean(axis=1)
                    extracted_data_scale = extracted_data.T-mean_base
                    # print(extracted_data_scale.shape)
                    extracted_data_norm = extracted_data_scale/extracted_data_scale.std() # z-score the trial
                    extracted_data_scale = extracted_data_scale.T
                    extracted_data_norm = extracted_data_norm.T
                    control_time = control_time-start_point +time_adjuster
                    # if control_time[0]>0:
                    if control_time>0:

                    
                        trials_dict['trial'].append(extracted_data_scale[:, 100:])
                        trials_dict['trial_norm'].append(extracted_data_norm[:, 100:])
                        trials_dict['baseline'].append(extracted_data_scale[:, :100])
                        trials_dict['baseline_norm'].append(extracted_data_norm[:, :100])
                        trials_dict['resp_point'].append(resp_point)
                        trials_dict['label'].append(cond)
                        trials_dict['confidence'].append(conf_val)
                        trials_dict['crosstime'].append(control_time[0])
                        # trials_dict['crosstime'].append(control_time)

                        
                
    
    
print(len(trials_dict['trial']),len(trials_dict['trial_norm']),len(trials_dict['baseline']),len(trials_dict['baseline_norm']),
      len(trials_dict['resp_point']), len(trials_dict['label']),len(trials_dict['confidence']),)
print(trials_dict['trial'][0].shape,trials_dict['trial_norm'][0].shape,trials_dict['baseline'][0].shape,trials_dict['baseline_norm'][0].shape)

In [ ]:
# # Example usage
# N_tr = np.sum(np.asarray(trials_dict['crosstime'])==None)

# list2 = np.array(np.asarray(trials_dict['resp_point'])[np.asarray(trials_dict['crosstime'])!=None]) # control
# list1 = np.asarray(trials_dict['resp_point'])[np.asarray(trials_dict['crosstime'])==None] # Face
# trials_dict['matching'] = np.full((len(list1)), -999)
# tol   = 600

# matches = match_with_tolerance(list1, list2, tol)
# for i, j in enumerate(matches):
#     if j is None:
#         print(f"list1[{i}] = {list1[i]:.2f} → no match")
#     else:
#         print(f"list1[{i}] = {list1[i]:.2f} ↔ list2[{j}] = {list2[j]:.2f}")

# for l1,l2 in enumerate(matches):
#     # print(l1,l2)
#     if l1!=None and l2!=None:
#         trials_dict['matching'][l1] = l2+N_tr
#         # trials_dict['matching'][l2+N_tr] = l1
    

# np.sum(trials_dict['matching']>=0)

#------------- QUELLO VERO DA DECOMMENTARE CON I TRIGGER GIUSTI -------------------------------------#


# Example usage
N_tr = np.sum(np.asarray(trials_dict['crosstime'])==None) # Number of face trials

list2 = np.array(np.asarray(trials_dict['resp_point'])[np.asarray(trials_dict['crosstime'])!=None]) # control
list2_rt = np.array(np.asarray(trials_dict['resp_point'])[np.asarray(trials_dict['crosstime'])!=None]) # control
list1 = np.asarray(trials_dict['resp_point'])[np.asarray(trials_dict['crosstime'])==None] # Face
trials_dict['matching'] = np.full((len(list1)), -999)
tol   = 600

matches = match_with_tolerance(list1, list2, tol)
for i, j in enumerate(matches):
    if j is None:
        print(f"list1[{i}] = {list1[i]:.2f} → no match")
    else:
        print(f"list1[{i}] = {list1[i]:.2f} ↔ list2[{j}] = {list2_rt[j]:.2f}")

for l1,l2 in enumerate(matches):
    # print(l1,l2)
    if l1!=None and l2!=None:
        trials_dict['matching'][l1] = l2+N_tr
        # trials_dict['matching'][l2+N_tr] = l1
    

np.sum(trials_dict['matching']>=0)

In [ ]:
print(trials_dict['matching'], len(trials_dict['matching']))

In [ ]:
# Print the non-matched trials and controls
print('Controls')
for i in range(N_tr):
    if i not in matches:
        print(trials_dict['crosstime'][N_tr:][i])
print('Face')
for i,m in enumerate(matches):
    if m==None:
        print(trials_dict['resp_point'][:N_tr][i])

In [ ]:
### Breaking ###
if breaking:
    with open('D:/PhD/CFS_eeg/data/new_exp/prep_task/breaking/subj_'+subj+'_trials_dict.pkl', 'wb') as handle:
        pickle.dump(trials_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Reverse ###
elif reverse:
    with open('D:/PhD/CFS_eeg/data/new_exp/prep_task/reverse/subj_'+subj+'_rev_trials_dict.pkl', 'wb') as handle:
        pickle.dump(trials_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
fig =plt.figure()
plt.hist(trials_dict['resp_point'][:142], alpha=0.4, bins=20);
plt.hist(np.array(trials_dict['crosstime'])[np.array(trials_dict['crosstime'])!=None], alpha=0.5, bins=20);
plt.show()

In [ ]:
subj